In [23]:
#memanggil library yang kita butuhkan dalam proses data cleaning
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams["figure.figsize"]= 20,10
%matplotlib inline


In [24]:
#membaca data dalam format csv yang tersimpan pada local directory
data1 = pd.read_csv("Price_House_Data.csv")
data1.head() #menampilkan 5 data teratas pada tabel

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [25]:
print('=>ini adalah info jumlah baris dan kolom dataset :', data1.shape)
print()
print('=>ini adalah info jumlah area_type pada dataset :', data1.groupby('area_type')['area_type'].agg('count'))

=>ini adalah info jumlah baris dan kolom dataset : (13320, 9)

=>ini adalah info jumlah area_type pada dataset : area_type
Built-up  Area          2418
Carpet  Area              87
Plot  Area              2025
Super built-up  Area    8790
Name: area_type, dtype: int64


In [26]:
#jika kita perhatikan ada beberapa variabel pada kolom yang pada dasarnya tidak terlalu penting seperti area_type,availability, society, balcony
#sehingga kita dapat buang, sehingga dari 9 kolom menjadi 5 kolom tersedia
data2 = data1.drop(['area_type', 'availability', 'society', 'balcony'],axis='columns')
data2.head()
data2.shape

(13320, 5)

In [27]:
#ketika kolom data kita hapus sekarang kita perlu check baris yang kemungkinan null dalam artian missing data/info
print('ini adalah jumlah data yang tidak tersedia :', data2.isnull().sum())
# data yang tidak tersedia dalam jumlah kecil jika dibadingkan keseluruhan data yang ada pada dataset
#sehingga kita dapat menghapus data yang tidak tersedia secara aman

ini adalah jumlah data yang tidak tersedia : location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64


In [28]:
#data yang tidak tersedia berhasil kita buang
data3 = data2.dropna()
data3.isnull().sum()
data2.shape

(13320, 5)

In [29]:
#setalah kita berhasil membersihkan kolom dan baris yang tidak kita butuhkan maka hal selanjutlah ada mengeksplor data lebih dalam
data3['size'].unique()
#kolom size : jika kita perhatikan ada info 2 BHK dst yang pada dasarnya info string sperti BHK dan bedroom tidak kita butuhkan kelak pada saat perhitungan sehingga hal tersebut kita dapat buang dan menginputkan dalam kolom tersendiri
data3['bhk'] = data3['size'].apply(lambda x : int(x.split(' ')[0]))
data3.head()

/home/rama/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,51.00,2


In [30]:
#jika diperhatikan jumlah bhk sangat bervariatif untuk setiap apartement yang ada sehingga kita dapat prediksi bahwa jika semakin banyak jumlah bhk maka akan semakin luas total_sqft(luas bangunan ya)
data3['bhk'].unique()

array([ 2,  4,  3,  6,  1,  8,  7,  5, 11,  9, 27, 10, 19, 16, 43, 14, 12,
       13, 18])

In [31]:
data3[data3.bhk>15]


,location,size,total_sqft,bath,price,bhk
1718,2Electronic City Phase II,27 BHK,8000,27.0,230.0,27
3379,1Hanuman Nagar,19 BHK,2000,16.0,490.0,19
3609,Koramangala Industrial Layout,16 BHK,10000,16.0,550.0,16
4684,Munnekollal,43 Bedroom,2400,40.0,660.0,43
11559,1Kasavanhalli,18 Bedroom,1200,18.0,200.0,18


In [32]:
#terlihat ada beberapa keganjilan dari prediksi yang kita lakukan. yakni baris 4648 bhk berjumlah 43 namun luas bangunan lebih kecil jika dibandingkan yang lain.
#sepertinya disini ada error pada tampilan data pada total_sqft sehingga kita butuh eksplor kolom total_sqft lebih dalam
data3['total_sqft'].unique()


array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [33]:
#tajuan saya disini adalah membuat data menjadi lebih seragam dan logis dalam artian data tunggal
#terlihat ada pola range seperti yang diperlihatkan pada data '1133 - 1348' yang kemungkinan bahwa data lain memiliki pola yang sama namu tidak tampil.
#maka kita coba dalami data tersebut
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [34]:
data3[~data3['total_sqft'].apply(is_float)]

,location,size,total_sqft,bath,price,bhk
30,Yelahanka,4 BHK,2100 - 2850,4.0,186.000,4
122,Hebbal,4 BHK,3067 - 8156,4.0,477.000,4
137,8th Phase JP Nagar,2 BHK,1042 - 1105,2.0,54.005,2
165,Sarjapur,2 BHK,1145 - 1340,2.0,43.490,2
188,KR Puram,2 BHK,1015 - 1540,2.0,56.800,2
...,...,...,...,...,...,...
12975,Whitefield,2 BHK,850 - 1060,2.0,38.190,2
12990,Talaghattapura,3 BHK,1804 - 2273,3.0,122.000,3
13059,Harlur,2 BHK,1200 - 1470,2.0,72.760,2
13265,Hoodi,2 BHK,1133 - 1384,2.0,59.135,2


In [35]:
#oke tahap selanjutnya adalah mencari nilai rata - rata dan membuang semua data yang ada string ya contoh pada baris 410
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return(float(tokens[0]) + float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [36]:
data4 = data3.copy()
data4['total_sqft']=data4['total_sqft'].apply(convert_sqft_to_num)
data4.head()
data4.shape

(13246, 6)

In [37]:
print('ini adalah jumlah data yang tidak tersedia :', data4.isnull().sum())
data4 = data4.dropna()
data4.isnull().sum()

ini adalah jumlah data yang tidak tersedia : location       0
size           0
total_sqft    46
bath           0
price          0
bhk            0
dtype: int64


location      0
size          0
total_sqft    0
bath          0
price         0
bhk           0
dtype: int64

In [38]:
#dari hasil proses data cleaning diatas membuat dataset menjadi lebih baik dari sebelumnya
#tahap terakhir dari data cleaning(optional) adalah saving data hasil dari proses data cleaning


In [39]:
data4.to_csv('PriceHouseData_Cleaning.csv', index=False)


In [40]:
data4 = pd.read_csv("PriceHouseData_Cleaning.csv")
data4.head(10)

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3
4,Kothanur,2 BHK,1200.0,2.0,51.00,2
5,Whitefield,2 BHK,1170.0,2.0,38.00,2
6,Old Airport Road,4 BHK,2732.0,4.0,204.00,4
7,Rajaji Nagar,4 BHK,3300.0,4.0,600.00,4
8,Marathahalli,3 BHK,1310.0,3.0,63.25,3
9,Gandhi Bazar,6 Bedroom,1020.0,6.0,370.00,6


In [41]:
len(data4.location.unique())

1298

In [42]:
data4.shape

(13200, 6)